<a href="https://colab.research.google.com/github/Tunyalucklie/lantech/blob/main/Project_Language_Tech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Sentiment analysis for content moderation in social media communities**




picture

brief project explanation

# Preprocessing
- split training and testing dataset
- build the bag of words representation and create dict to store sentence information (words and classes)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import math

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
path="/content/drive/MyDrive/lang tech/"
os.chdir(path)

Dataset information: .....

In [ ]:
data = pd.read_csv("HateSpeechDataset.csv",encoding='utf-8')

In [ ]:
os.listdir()

['HateSpeechDatasetBalanced.csv', 'HateSpeechDataset.csv']

In [ ]:
data

,Content,Label,Content_int
0,denial of normal the con be asked to comment o...,1,"[146715, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,..."
1,just by being able to tweet this insufferable ...,1,"[146715, 14, 15, 16, 17, 7, 18, 19, 20, 21, 22..."
2,that is retarded you too cute to be single tha...,1,"[146715, 28, 29, 30, 26, 31, 32, 7, 5, 33, 28,..."
3,thought of a real badass mongol style declarat...,1,"[146715, 35, 1, 24, 36, 37, 38, 39, 40, 1, 41,..."
4,afro american basho,1,"[146715, 46, 47, 48, 146714]"
...,...,...,...
440900,crash another movie from left field i have to ...,0,"[146715, 1780, 649, 1673, 347, 701, 1702, 80, ..."
440901,i why do not you debate first before starting ...,0,"[146715, 80, 186, 340, 76, 26, 6407, 566, 2608..."
440902,removal of i reverted the removal of the above...,0,"[146715, 18731, 1, 80, 48182, 3, 18731, 1, 3, ..."
440903,i have unblocked you eddie as i discussed on u...,0,"[146715, 80, 87, 20515, 26, 10684, 273, 80, 95..."


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440905 entries, 0 to 440904
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Content      440905 non-null  object
 1   Label        440905 non-null  object
 2   Content_int  440905 non-null  object
dtypes: object(3)
memory usage: 10.1+ MB


In [ ]:
data.head(5)

,Content,Label,Content_int
0,denial of normal the con be asked to comment o...,1,"[146715, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,..."
1,just by being able to tweet this insufferable ...,1,"[146715, 14, 15, 16, 17, 7, 18, 19, 20, 21, 22..."
2,that is retarded you too cute to be single tha...,1,"[146715, 28, 29, 30, 26, 31, 32, 7, 5, 33, 28,..."
3,thought of a real badass mongol style declarat...,1,"[146715, 35, 1, 24, 36, 37, 38, 39, 40, 1, 41,..."
4,afro american basho,1,"[146715, 46, 47, 48, 146714]"


In [ ]:
data["Content"][0].split()

['denial',
 'of',
 'normal',
 'the',
 'con',
 'be',
 'asked',
 'to',
 'comment',
 'on',
 'tragedies',
 'an',
 'emotional',
 'retard']

In [ ]:
#split the train data and test data

x, y = data['Content'] , data['Label']
train_split = int(0.7 * x.shape[0])
x_train, y_train = x[:train_split], y[:train_split]
x_test, y_test = x[train_split:], y[train_split:]
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((308633,), (308633,), (132272,), (132272,))

In [ ]:
#make data structure
# zero is positive , one is negative
train_dict = [{'words': sentence.split(), 'class': 'neg' if label == 1 else 'pos'} for sentence, label in zip(x_train, y_train)]
test_dict = [{'words': sentence.split(), 'class': 'neg' if label == 1 else 'pos'} for sentence, label in zip(x_test, y_test)]

print(train_dict[0])
print(test_dict[0])

{'words': ['denial', 'of', 'normal', 'the', 'con', 'be', 'asked', 'to', 'comment', 'on', 'tragedies', 'an', 'emotional', 'retard'], 'class': 'pos'}
{'words': ['kicking', 'off', 'the', 'european', 'vacation', 'tour', 'at', 'buckinghampalace'], 'class': 'pos'}


In [ ]:
#build data in pandas format
# content | bag of words | class

# Analysis on Sentence, Words and Class

# System Development
- training models including NaiveBayes clssifer, SVM and LSTM
- Intrinsic evalution on each models
- optimization on each models


# NaiveBayes Model

In class method (lab1)

In [ ]:
class NaiveBayes(object):

    def predict(self, d): #d is data, represent a sentense
        # TODO: Replace the following line with your own code to solve Problem 2
        pr_P=0 #positive
        pr_N=0 #negative
        for word in d:
            if word in self.vocabulary:
                pr_P=pr_P+self.pw['pos'][word] #pw means Pr(w|c)
                pr_N=pr_N+self.pw['neg'][word]
            else:
                pass
        pr_P=pr_P+self.pc['pos']
        pr_N=pr_N+self.pc['neg']
        # print("predict finished")
        if(pr_P>pr_N):
            return 'pos'
        else:
            return 'neg'


    @classmethod
    def train(cls, data, k=1):
        # The following line creates a new object of type OurClassifier:
        classifier = cls()
        # The next few lines initialise the four attributes of the classifier:
        classifier.classes = set()
        classifier.vocabulary = set()
        classifier.pc = {}
        classifier.pw = {}

        # TODO: Insert code to solve Problem 2

        for i in range(len(data)): #create classes and count occurance numbers
            newclass=data[i]['class']
            if newclass not in classifier.classes:
                classifier.classes.add(newclass) #add newclass
                classifier.pc[newclass]=1        #initialize the occurance number of this new class
                classifier.pw[newclass]={}        #create the new class dictionary of probability

            else:
                classifier.pc[newclass]+=1       #increment occurrence count of the existing class by 1

        for classes in classifier.pc: #calculate probabilities of classes
            classifier.pc[classes]=math.log(classifier.pc[classes]/len(data))

        for i in range(len(data)): #create vocabulary and count occurance numbers
            thisclass=data[i]['class']
            sentence=data[i]['words']
            for words in sentence:
                if words not in classifier.vocabulary: #if we find a new word
                    classifier.vocabulary.add(words)
                    for c in classifier.classes:
                        classifier.pw[c][words]=k
                    classifier.pw[thisclass][words]+=1
                else:
                    classifier.pw[thisclass][words]+=1
        for c in classifier.pw:
            total=sum(list(classifier.pw[c].values()))
            for word in classifier.pw[c]:
                classifier.pw[c][word]=math.log(classifier.pw[c][word]/total)

        return classifier

In [ ]:
classifier=NaiveBayes.train(train_dict)

In [ ]:
classifier.pw["pos"]["denial"]

-10.022640035110399

In [ ]:
#evaluation in-class method

#yeokwoon...

outsource libralies

In [ ]:
#Aom

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# Model training
model.fit(X_train, y_train)

# Predict Output
predicted = model.predict([X_test[6]])

# SVM Model

In [ ]:
#yeokwoon

# LSTM Model

In [ ]:
#Aom

# Systems Evaluation and Result Analysis

Testing model with xxxx data
This data is from ......

**Comparison Performance on Different Models**

performance on testing dataset

performance on xxx dataset

This illustration compares the final results

Final model will be : ....

# Model Deployment

In [ ]:
# model input - prompt